In [4]:
import json
import os
import math
import librosa
import boto3
import soundfile as sf
from pydub import AudioSegment
from app import *
from flask_mysqldb import MySQL
from io import BytesIO
import io
from werkzeug.utils import secure_filename
from six.moves.urllib.request import urlopen

c:\Users\OOi QI HAO\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [24]:
import mysql.connector
conn = mysql.connector.connect(user='9rBnbH2yWN', password='obqUHk85vy', host='remotemysql.com', database='9rBnbH2yWN')
cur = conn.cursor()
query = 'SELECT s3_obj_url FROM test WHERE normal IS NULL'
cur.execute(query)
records = cur.fetchall()
print(records)

[('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/2022-05-12T08_22_39.866Z.wav',), ('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000024.wav',), ('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000089.wav',), ('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000050.wav',), ('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000038.wav',), ('https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000021.wav',)]


In [25]:
BUCKET_NAME = "idp2g7"
ACCESS_KEY_ID = ""
SECRET_ACCESS_KEY = ""
s3 = boto3.resource('s3',aws_access_key_id= ACCESS_KEY_ID,aws_secret_access_key= SECRET_ACCESS_KEY)
bucket = s3.Bucket(BUCKET_NAME)
location = "ap-southeast-1"
for row in records:
    
    for obj in bucket.objects.filter(Prefix="test/"):
        obj_key = obj.key
        obj_url = "https://%s.s3.%s.amazonaws.com/%s" % ('idp2g7', location, obj_key)
        
        if obj_url==row[0]:
            print(obj_key)
            s3_client = boto3.client('s3',aws_access_key_id= ACCESS_KEY_ID,aws_secret_access_key= SECRET_ACCESS_KEY)
            object = s3_client.get_object(Bucket=BUCKET_NAME, Key=obj_key)
            data = io.BytesIO(object['Body'].read())
            audio = AudioSegment.from_wav(data)
            if audio.duration_seconds>5.0:
               first_5_seconds = audio[:5000]
               first_5_seconds.export(data, format="wav")
               print(first_5_seconds)
               s3_client.put_object(Body=data, Bucket= BUCKET_NAME, Key=obj_key)

test/2022-05-12T08_22_39.866Z.wav
test/00000024.wav
test/00000089.wav
test/00000050.wav
test/00000038.wav
test/00000021.wav


In [26]:
JSON_PATH = "data_nolabel.json" #change ur json file name here

SAMPLE_RATE = 22050
TRACK_DURATION = 5  # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION #=220500

In [27]:
def save_mfcc(json_path, num_mfcc=13, n_fft=2048, hop_length=512,num_segments=5):
    """Extracts MFCCs from audio and saves them into a json file
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mfcc": [],
        "obj_url": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments) #=44100
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment/ hop_length) #=87
    
    # loop through all genre sub-folder
    for row in records:
        for obj in bucket.objects.filter(Prefix="test/"):
            obj_key = obj.key
            obj_url = "https://%s.s3.%s.amazonaws.com/%s" % ('idp2g7', location, obj_key)
    
        
            if row[0]==obj_url:
                url = "https://s3-%s.amazonaws.com/%s/%s" % (location, 'idp2g7', obj_key)
                print(url)
                signal, sample_rate = sf.read(io.BytesIO(urlopen(url).read()),dtype='float32')
                signal = signal.T
                signal = librosa.resample(signal, orig_sr=16000, target_sr=SAMPLE_RATE)
                signal = librosa.to_mono(signal)
                sample_rate = SAMPLE_RATE

                for d in range(num_segments):
                # calculate start and finish sample for current segment
                    start = samples_per_segment*d
                    finish = start+samples_per_segment

                # extract mfcc
                    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                                hop_length=hop_length)
                    mfcc = mfcc.T

                # store only mfcc feature with expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["obj_url"].append(obj_url)
                        print("{},segment:{}".format(row[0],d+1))
                        print(mfcc.shape)

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)


if __name__ == "__main__":
    save_mfcc(JSON_PATH)
    



https://s3-ap-southeast-1.amazonaws.com/idp2g7/test/2022-05-12T08_22_39.866Z.wav


<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.0036261  -0.00415959
 -0.00404576], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[-0.00378364 -0.00410711 -0.00435738 ...  0.00616344  0.00568677
  0.0053805 ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[0.0058974  0.00646743 0.00675966 ... 0.01187905 0.01189297 0.01150639], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_f

https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/2022-05-12T08_22_39.866Z.wav,segment:1
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/2022-05-12T08_22_39.866Z.wav,segment:2
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/2022-05-12T08_22_39.866Z.wav,segment:3
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/2022-05-12T08_22_39.866Z.wav,segment:4
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/2022-05-12T08_22_39.866Z.wav,segment:5
(44, 13)
https://s3-ap-southeast-1.amazonaws.com/idp2g7/test/00000024.wav


<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[0.00169874 0.0015052  0.00063893 ... 0.0049383  0.00393412 0.00290541], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[0.00229612 0.00194866 0.0016574  ... 0.00101963 0.00137369 0.00188979], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[ 0.00226229  0.00255255  0.00262178 ... -0.00321089 -0.00331741
 -0.00327143], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_ff

https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000024.wav,segment:1
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000024.wav,segment:2
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000024.wav,segment:3
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000024.wav,segment:4
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000024.wav,segment:5
(44, 13)
https://s3-ap-southeast-1.amazonaws.com/idp2g7/test/00000089.wav


...

<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[-1.8637506e-03 -1.8744306e-03  5.2148069e-05 ... -3.5190918e-03
 -2.9808735e-03 -2.6583723e-03], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[-3.0026580e-03 -3.6382629e-03 -3.8351233e-03 ... -1.5062601e-03
 -5.9976446e-05  1.0431972e-03], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[-7.2533148e-05 -8.1385206e-04 -1.3928491e-04 ...  1.7126806e-03
  1.4865132e-03  3.5644732e-03], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature

https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000050.wav,segment:1
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000050.wav,segment:2
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000050.wav,segment:3
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000050.wav,segment:4
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000050.wav,segment:5
(44, 13)
https://s3-ap-southeast-1.amazonaws.com/idp2g7/test/00000038.wav


<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[-0.0002498  -0.00042269 -0.00052396 ...  0.00302231  0.00259873
  0.00271545], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[ 0.0025398   0.00251675  0.00304841 ...  0.00025461  0.00018576
 -0.00014587], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[-0.00030832  0.00017003  0.00043446 ...  0.0012856   0.00108846
  0.00090159], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mf

https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000038.wav,segment:1
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000038.wav,segment:2
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000038.wav,segment:3
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000038.wav,segment:4
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000038.wav,segment:5
(44, 13)
https://s3-ap-southeast-1.amazonaws.com/idp2g7/test/00000021.wav
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000021.wav,segment:1
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000021.wav,segment:2
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000021.wav,segment:3
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000021.wav,segment:4
(44, 13)
https://idp2g7.s3.ap-southeast-1.amazonaws.com/test/00000021.wav,segment:5
(44, 13)


<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[ 0.00089866 -0.00067046 -0.00205141 ...  0.0025769   0.00272186
  0.00304931], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[0.00324064 0.0029401  0.00256705 ... 0.000474   0.00030398 0.00024171], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
<ipython-input-27-a12b01ee5773>:42: FutureWarning: Pass y=[ 0.00062823  0.00031399 -0.00052389 ...  0.00137182  0.00212436
  0.00312665], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_f

In [9]:
import mysql.connector

def save_prediction(file_name,prediction): # fileneame must exist in database
    conn = mysql.connector.connect(user='9rBnbH2yWN', password='obqUHk85vy', host='remotemysql.com', database='9rBnbH2yWN')
    cursor = conn.cursor()
    cursor.execute("UPDATE test SET normal = %s ,fault = %s WHERE filename = %s",(prediction[0],prediction[1],file_name))
    conn.commit()
    conn.close()

# normal col(normal:0, abnormal:1), fault col(type 1-5,0 for no fault or "normal")
prediction_example = [1,3]
save_prediction("00000025.wav",prediction_example)


In [20]:
import numpy as np
myprediction = [4,4,5,5,4,9,8,9,9,8]
x = len(myprediction)/5
myprediction = np.array_split(myprediction, x)
print(myprediction[0])
print(np.bincount(myprediction[1]).argmax())

[4 4 5 5 4]
9


In [30]:
conn = mysql.connector.connect(user='9rBnbH2yWN', password='obqUHk85vy', host='remotemysql.com', database='9rBnbH2yWN')
cursor = conn.cursor()
cursor.execute("SELECT filename FROM test WHERE normal IS NULL") 
records = cursor.fetchall()
#print(records)
#print(str(records[0][0]))

for row in records:
    print(row[0])

2022-05-12T08_22_39.866Z.wav
00000024.wav
00000089.wav
00000050.wav
00000038.wav
00000021.wav
